In [ ]:
%load_ext watermark


In [ ]:
import os

import covasim as cv
import numpy as np
from teeplot import teeplot as tp

import pylib  # noqa: F401


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = os.environ.get("2025-03-31-dev-variantsync", None)
teeplot_subdir


## Within-host Dynamics


In [ ]:
pop_size = 10_000
num_variants = 5  # 0th variant is wild-type


In [ ]:
class SyncWithinHost:

    _host_compartments: np.ndarray

    def __init__(
        self: "SyncWithinHost", pop_size: int, num_variants: int
    ) -> None:
        shape = (pop_size, num_variants)
        self._host_compartments = np.zeros(shape, dtype=float)

    def __call__(self: "SyncWithinHost", sim: cv.Sim) -> None:
        compartments = self._host_compartments
        people = sim.people
        random_p = np.random.rand(*people["infectious_variant"].shape)

        ## sync sim to host compartments
        #######################################################################
        # zero out non-infectious/exposed compartments
        mask = ~(people["infectious"] | people["exposed"])
        compartments[mask, :] = 0.0

        # ensure host compartments are initialized
        for variant in range(1, num_variants):
            compartments[:, variant] = np.maximum(
                people["infectious_variant"] == variant,
                compartments[:, variant],
            )

        # update host compartments
        #######################################################################
        # grow strains
        compartments[:, 1] *= 2.0
        compartments[:, 2] *= 3.0  # lower-trans variant, faster growth
        compartments[:, 3] *= 2.0
        compartments[:, 4] *= 3.0  # lower-trans variant, faster growth

        # introduce low-transmissibility variants
        for lowtran_variant in 2, 4:
            p = 1.0 - np.power(0.99, compartments[:, lowtran_variant - 1])
            compartments[:, lowtran_variant] = np.maximum(
                random_p < p,
                compartments[:, lowtran_variant],
            )

        ## sync host to sim compartments
        #######################################################################
        compartments_ = compartments.copy()
        compartments_ *= np.random.rand(*compartments.shape)
        compartments_[1, :] *= 1.5  # higher-transmissiblity bonus
        compartments_[3, :] *= 1.5  # higher-transmissiblity bonus

        sampled_strains = np.where(
            compartments_.any(axis=1),
            np.argmax(compartments_, axis=1),
            np.nan,
        )

        people["infectious_variant"] = np.where(
            ~np.isnan(people["infectious_variant"]),
            sampled_strains,
            people["infectious_variant"],
        )


In [ ]:
sim = cv.Sim(
    interventions=[SyncWithinHost(pop_size, num_variants)],
    pop_infected=0,  # disable wild-type strain
    pop_size=pop_size,
    variants=[gamma, gamma_, delta, delta_],
    rand_seed=1,
)
sim.run()
tp.tee(
    sim.plot,
    "variant",
    teeplot_outattrs={"what": "within-host"},
)
...
